In [1]:
%%javascript
$('#appmode-leave').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Determination of the Rate Law \#1
You are studying the rate of decomposition of a reaction by monitoring the concentration of the reactant with time. 
Collect enough data to determine the order of the reaction, the rate constant, and the half life.

Note: every time you "Restart the laboratory" the concentration of the reactant may change.

### Instructions:

- Use the slide bar below to select the initial amount of $C_6H_5N_2Cl$ and the times at which you collect the amount of $N_2(g)$ produced, 
- Click `Perform measurement` to run the virtual experiment and obtain the result of the experiment,
- Click `Download CSV` to export the complete data set for all the experiments as a CSV file.

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

In [3]:
# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin4"]


In [4]:
# delete existing result file and setup rng
if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
    os.remove(os.path.join(os.getcwd(), "results.csv"))

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

species_A = [1]
def init():
    global species_A 
    species_A[0] = random.random()


In [5]:
out_R = ipw.Label(value="Click Calculate.")
out_P = ipw.Output()
out_L = ipw.Output()

result = "Concentration (mol/L)"
value = "Time (s)"

with out_L:
    display(Markdown("[Download CSV](./results.csv)"))
    
def calc(btn):
    c = (1. / species_A[0]) + params["k"] * time.value
    R = 1. / c
    
    # Random error
    dR = R * params["error"].get("random", 0) * (0.5 - random.random()) * 2 
    # Systematic error
    dR = dR + R * params["error"].get("systematic", 0)
    out_R.value = f"{R + dR:.6e}"
    out_P.clear_output()
    try:
        res = pd.read_csv("results.csv") 
    except FileNotFoundError:
        res = pd.DataFrame(columns=[value,
                                    result])
    res = res.append({value: time.value, 
                      result: R + dR}, 
                     ignore_index=True)
    res.to_csv("results.csv", index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
        os.remove(os.path.join(os.getcwd(), "results.csv"))
    res = pd.DataFrame(columns=[value,
                                result])
    res.to_csv("results.csv", index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(50))
    init()

time = ipw.FloatLogSlider(value=params["time"], min=0, max=5)

btn_reset = ipw.Button(description="Restart Laboratory", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)


btn_calc = ipw.Button(description="Perform measurement", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

rows = []
rows.append(ipw.HBox([ipw.Label(value=f"Elapsed time:", 
                                layout=ipw.Layout(width="250px")),
                      time,
                      ipw.Label(value="(s)")]))

rows.append(ipw.HBox([ipw.Label(value="Concentration:", 
                                layout=ipw.Layout(width="250px")), 
                      out_R,
                      ipw.Label(value="mol/L")]))

rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))
rows.append(ipw.HBox([out_P]))
reset(btn_reset)
ipw.VBox(rows)